In [1]:
from PIL import Image
import numpy as np
import torch

from DeepPrint.model import DeepPrintNet

In [2]:
model = DeepPrintNet()

In [3]:
sample_input = torch.randn(1, 1, 448, 448)
output = model(sample_input)

In [4]:
embedding, map, aligment, aligned = output.values()
print('embedding shape:', embedding.shape)
print('map shape:', map.shape)
print('aligment shape:', aligment.shape)
print('aligned shape:', aligned.shape)

embedding shape: torch.Size([1, 192])
map shape: torch.Size([1, 96])
aligment shape: torch.Size([1, 3])
aligned shape: torch.Size([1, 1, 448, 448])


In [5]:
create_model_image = False

if create_model_image:
    from torchviz import make_dot

    sample_input = torch.randn(1, 1, 448, 448)
    output = model(sample_input)
    make_dot(output['embedding'], params=dict(model.named_parameters())).render("deepprint_graph", format="png")
